Import libraries

In [2]:
import pandas as pd
import numpy as np
!pip install lxml


Download data and preprocess it

In [9]:
path = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M" #Set path
read = pd.read_html(path) # Read the file in list format

In [47]:
df = pd.DataFrame(read[0]) #convert the file into dataframe
df = df[df.Borough != 'Not assigned'] # exclude lines with no Borough data
df['Neighborhood'] = df['Neighborhood'].replace('Not assigned', df['Borough']) #put borough in "empty" neoghbourhood cells
df.reset_index(drop=True, inplace=True) # reset index
df.head(12)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [37]:
df.shape #check size of the dataframe

(103, 3)

In [41]:
geo = pd.read_csv('http://cocl.us/Geospatial_data')

In [48]:
dfg = pd.DataFrame(geo)
df['Latitude'] = dfg['Latitude']
df['Longitude'] = dfg['Longitude']
df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.806686,-79.194353
1,M4A,North York,Victoria Village,43.784535,-79.160497
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.763573,-79.188711
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.770992,-79.216917
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.773136,-79.239476


Download other libraries

In [49]:
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: - 
  - anaconda/osx-64::openssl-1.1.1d-h1de35cc_2
  - defaults/osx-64::openssl-1.1.1d-h1de35ccdone

## Package Plan ##

  environment location: /opt/anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.22.0-pyh9f0ad1d_0



geographiclib-1.50   | 34 KB     | ##################################### | 100% 
geopy-1.22.0         | 63 KB     | ##################################### | 100% 
Prepa

In [66]:
dfs = df[df['Borough'].astype(str).str.contains('Toronto')]
dfs.reset_index(drop=True, inplace=True) # reset index
dfs.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.763573,-79.188711
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.773136,-79.239476
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.692657,-79.264848
3,M5C,Downtown Toronto,St. James Town,43.799525,-79.318389
4,M4E,East Toronto,The Beaches,43.786947,-79.385975


Explore the data

In [54]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 43.6534817, -79.3839347.


Create a map of Toronto with neighborhoods superimposed on top.

In [55]:
# create map of Toronto using latitude and longitude values
map_tor = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(dfs['Latitude'], dfs['Longitude'], dfs['Borough'], dfs['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tor)  
    
map_tor

Define Foursquare Credentials and Version

In [56]:
CLIENT_ID = 'OOEZR4SAHZBM0NNFSRYYTXNVCBXVTQ4ZFHRZKDPNE04V110L' # your Foursquare ID
CLIENT_SECRET = 'SBJ1SZCU0YFJ0JWBAC5JFPLIMH3NDZNFFQ3B3NQL1G2YXHQC' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: OOEZR4SAHZBM0NNFSRYYTXNVCBXVTQ4ZFHRZKDPNE04V110L
CLIENT_SECRET:SBJ1SZCU0YFJ0JWBAC5JFPLIMH3NDZNFFQ3B3NQL1G2YXHQC


Explore the first neighborhood in our dataframe

In [70]:
n_latitude = dfs.loc[0, 'Latitude'] # neighborhood latitude value
n_longitude = dfs.loc[0, 'Longitude'] # neighborhood longitude value

n_name = dfs.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(n_name, 
                                                               n_latitude, 
                                                               n_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.7635726, -79.1887115.


Get the top 100 venues that are in Marble Hill within a radius of 500 meters

In [71]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # radius of 500 meters

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    latitude, 
    longitude, 
    VERSION, 
    radius, 
    LIMIT)

results = requests.get(url).json()

In [72]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [73]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Downtown Toronto,Neighborhood,43.653232,-79.385296
1,Nathan Phillips Square,Plaza,43.652270,-79.383516
2,Indigo,Bookstore,43.653515,-79.380696
3,Chatime 日出茶太,Bubble Tea Shop,43.655542,-79.384684
4,Textile Museum of Canada,Art Museum,43.654396,-79.386500


In [74]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

77 venues were returned by Foursquare.


In [75]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [78]:
dfs_venues = getNearbyVenues(names=dfs['Neighborhood'],
                                   latitudes=dfs['Latitude'],
                                   longitudes=dfs['Longitude']
                                  )
dfs_venues.head()

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
1,"Regent Park, Harbourfront",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
2,"Regent Park, Harbourfront",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant
3,"Regent Park, Harbourfront",43.763573,-79.188711,Enterprise Rent-A-Car,43.764076,-79.193406,Rental Car Location
4,"Regent Park, Harbourfront",43.763573,-79.188711,Woburn Medical Centre,43.766631,-79.192286,Medical Center


In [80]:
print('There are {} uniques categories.'.format(len(dfs_venues['Venue Category'].unique())))

There are 207 uniques categories.


Analyze Each Neighborhood

In [81]:
# one hot encoding
dfs_onehot = pd.get_dummies(dfs_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
dfs_onehot['Neighborhood'] = dfs_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [dfs_onehot.columns[-1]] + list(dfs_onehot.columns[:-1])
dfs_onehot = dfs_onehot[fixed_columns]

dfs_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,...,Thrift / Vintage Store,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [82]:
dfs_onehot.shape

(754, 207)

Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [84]:
dfs_grouped = dfs_onehot.groupby('Neighborhood').mean().reset_index()
dfs_grouped

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Thrift / Vintage Store,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0000,0.000000,0.000000,0.000000,0.00,0.000,0.00,0.000000,0.0000,0.00
1,"Brockton, Parkdale Village, Exhibition Place",0.024390,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.048780,0.000000,...,0.0000,0.000000,0.000000,0.000000,0.00,0.000,0.00,0.024390,0.0000,0.00
2,"Business reply mail Processing Centre, South C...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0000,0.000000,0.000000,0.000000,0.00,0.000,0.00,0.000000,0.0000,0.00
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.062500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0000,0.000000,0.000000,0.000000,0.00,0.000,0.00,0.000000,0.0000,0.00
4,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0000,0.000000,0.000000,0.000000,0.00,0.000,0.00,0.000000,0.0000,0.00
5,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0000,0.000000,0.000000,0.000000,0.00,0.000,0.00,0.000000,0.0000,0.00
6,Church and Wellesley,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0000,0.000000,0.000000,0.000000,0.00,0.000,0.00,0.000000,0.0000,0.00
7,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0000,0.000000,0.000000,0.000000,0.00,0.000,0.00,0.000000,0.0000,0.00
8,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0000,0.000000,0.200000,0.000000,0.00,0.000,0.00,0.000000,0.0000,0.00
9,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0000,0.000000,0.000000,0.050000,0.00,0.000,0.05,0.016667,0.0000,0.00


Print each neighborhood along with the top 5 most common venues

In [86]:
num_top_venues = 5

for hood in dfs_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = dfs_grouped[dfs_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                       venue  freq
0                  Cafeteria   1.0
1                Yoga Studio   0.0
2         Miscellaneous Shop   0.0
3                       Lake   0.0
4  Latin American Restaurant   0.0


----Brockton, Parkdale Village, Exhibition Place----
                 venue  freq
0                 Café  0.10
1          Coffee Shop  0.07
2              Brewery  0.05
3            Gastropub  0.05
4  American Restaurant  0.05


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
               venue  freq
0        Pizza Place  0.25
1     Sandwich Place  0.25
2           Bus Line  0.25
3  Mobile Phone Shop  0.25
4        Yoga Studio  0.00


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                venue  freq
0         Yoga Studio  0.06
1          Skate Park  0.06
2       Garden Center  0.06
3              Garden  0.06
4  Light Rail Station  0.06

Put that into a pandas dataframe

In [92]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
n_venues_sorted = pd.DataFrame(columns=columns)
n_venues_sorted['Neighborhood'] = dfs_grouped['Neighborhood']

for ind in np.arange(dfs_grouped.shape[0]):
    n_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dfs_grouped.iloc[ind, :], num_top_venues)

n_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Cafeteria,Women's Store,Dessert Shop,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Gastropub,Brewery,Bakery,American Restaurant,Yoga Studio,Middle Eastern Restaurant,Bookstore,Pet Store
2,"Business reply mail Processing Centre, South C...",Bus Line,Sandwich Place,Pizza Place,Mobile Phone Shop,Women's Store,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store
3,"CN Tower, King and Spadina, Railway Lands, Har...",Yoga Studio,Auto Workshop,Park,Pizza Place,Butcher,Light Rail Station,Burrito Place,Restaurant,Brewery,Farmers Market
4,Central Bay Street,Coffee Shop,Grocery Store,Pharmacy,Pizza Place,Bank,Butcher,Department Store,Dumpling Restaurant,Donut Shop,Doner Restaurant


Run k-means to cluster the neighborhood into 5 clusters.

In [98]:
# set number of clusters
kclusters = 5

dfs_grouped_clustering = dfs_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dfs_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

# add clustering labels
n_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

dfs_merged = dfs

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
dfs_merged = dfs_merged.join(n_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

dfs_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.763573,-79.188711,3,Intersection,Breakfast Spot,Bank,Mexican Restaurant,Rental Car Location,Medical Center,Electronics Store,Women's Store,Diner,Dumpling Restaurant
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.773136,-79.239476,3,Bakery,Hakka Restaurant,Bank,Gas Station,Athletics & Sports,Caribbean Restaurant,Fried Chicken Joint,Thai Restaurant,Diner,Women's Store
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.692657,-79.264848,3,College Stadium,General Entertainment,Café,Skating Rink,Dessert Shop,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
3,M5C,Downtown Toronto,St. James Town,43.799525,-79.318389,3,Chinese Restaurant,Electronics Store,Grocery Store,Breakfast Spot,Burger Joint,Fast Food Restaurant,Bank,Pharmacy,Gym Pool,Gym
4,M4E,East Toronto,The Beaches,43.786947,-79.385975,3,Chinese Restaurant,Bank,Café,Japanese Restaurant,Women's Store,Dim Sum Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


Visualize

In [99]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dfs_merged['Latitude'], dfs_merged['Longitude'], dfs_merged['Neighborhood'], dfs_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Examine the first cluster and determine the discriminating venue categories that distinguish it

In [100]:
dfs_merged.loc[dfs_merged['Cluster Labels'] == 0, dfs_merged.columns[[1] + list(range(5, dfs_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,West Toronto,0,Shopping Mall,Park,Grocery Store,Bank,Dessert Shop,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
26,Central Toronto,0,Construction & Landscaping,Park,Trail,Basketball Court,Bakery,Women's Store,Dim Sum Restaurant,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
28,West Toronto,0,Convenience Store,Pizza Place,Caribbean Restaurant,Women's Store,Dessert Shop,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
